In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
import numpy as np
import requests
import re
import random

In [2]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
def get_property_urls(link):
    # omg thank god for some dude on github for this req_header 0 CAPTCHAS i owe you my life
    
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }


    r = requests.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    table = soup.find('ul', class_='photo-cards photo-cards_wow photo-cards_short')
    li_tags = table.find_all('li')
    properties = [item.find('div', class_='list-card-info')for item in li_tags]

    property_urls = []

    for item in properties:
        if item != None:
            property_extensions = item.find('a')['href']
            property_urls.append(property_extensions)
            
    return property_urls

In [4]:
def get_property_features(link):
    
    """
    Description: Grab all features from a listings URL (HOMES FOR SALE)
    Parameters: Link to listing (str)
    Return: all features from that properties url (dict)
    """
    
    try:
    
        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }

        r = requests.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')

        # this is the code for extracting features our of the home fact list table
        # creating a features list variable that we will append each scraped feature to
        # I'm deciding to do this so we can pull all the data in a particular table 
        # all at once and just use their entries for our column names
        
        all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
        features_list = [item.text.split(':') for item in all_features]
        address = soup.find('h1', class_='ds-address-container').text.replace(',\xa0', ', ')
        price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
        summary_info = soup.find_all('span', class_='ds-bed-bath-living-area')

        # getting the third element in summnary info which is square feet and then splitting/
        # reversing it so its formatted as ['Square Feet', '728'] and not ['728', 'square feet']
        for item in summary_info:
            if 'sqft' in item.text.lower() or 'square feet' in item.text.lower():
                sqft = item.text.split(" ", 1)[::-1]
            else:
                sqft = np.nan

        # trying neighborhood stats in new way if this doesnt work uncomment code above
        neighborhood_stats_formatted = soup.find(text = 'Neighborhood stats').next_element.get_text()

        # finding if building was recently constructed or renovated from construction details
        # table 
        construction_details = soup.find_all('span', class_='Text-aiai24-0 cZksOw')
        for item in construction_details:
            if 'new construction' in item.text.lower():
                construction = item.text.split(':')

        features_list.append(['Total Price', price])
        features_list.append(['Address', address])
        features_list.append(sqft)
        features_list.append(construction)

        # this is code for extracting number of bedroom/baths from a separate div
        # then formatting it to match our features from the first table and appending
        # to feature list
        beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
        beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
        bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

        for item in bed_baths_formatted:
            features_list.append(item.split(':'))

        features_list.append(['neighborhood_stats', neighborhood_stats_formatted])
        features_list.append(['URL', link])
        
        print(features_list)
        return dict(features_list)
    except:
        return None

In [5]:

def get_property_features_sold(link):
    
    """
    Description: Grab all features from a listings URL (HOMES SOLD)
    Parameters: Link to listing (str)
    Return: all features from that properties url (dict)
    """
        
    try:
    
        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }


        r = requests.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


        # this is the code for extracting features our of the home fact list table
        all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
        features_list = [item.text.split(':') for item in all_features]
        address = soup.find('h1', class_='ds-address-container').text.replace(',\xa0', ', ')
        price = soup.find('span', class_='sc-pscky hEQFyS ds-status-details').get_text().lower().split(':')
        
        # so zillow has homes sold filter but people flip properties very quickly i guess
        # anyway we dont want to grab duplicates homes from when we looked at only for sale homes
        # so im creating this condition to make sure the status bar for the home says "Sold"
        
        if 'sold' in price:

            summary_info = soup.find_all('span', class_='ds-bed-bath-living-area')

            # getting the third element in summnary info which is square feet and then splitting/
            # reversing it so its formatted as ['Square Feet', '728'] and not ['728', 'square feet']
            for item in summary_info:
                if 'sqft' in item.text.lower() or 'square feet' in item.text.lower():
                    sqft = item.text.split(" ", 1)[::-1]
                else:
                    sqft = np.nan

            # getting all text out of neighborhoods stats about market trends 
            neighborhood_stats_formatted = soup.find(text = 'Neighborhood stats').next_element.get_text()

            # finding if building was recently constructed or renovated from construction details
            construction_details = soup.find_all('span', class_='Text-aiai24-0 cZksOw')
            for item in construction_details:
                if 'new construction' in item.text.lower():
                    construction = item.text.split(':')

            features_list.append(['Total Price', price[1]])
            features_list.append(['Address', address])
            features_list.append(sqft)
            features_list.append(construction)

            # this is code for extracting number of bedroom/baths from a separate div
            # then formatting it to match our features from the first table and appending
            # to feature list
            beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
            beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
            bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

            for item in bed_baths_formatted:
                features_list.append(item.split(':'))

            features_list.append(['neighborhood_stats', neighborhood_stats_formatted])
            features_list.append(['URL', link])

            print(features_list)
            return dict(features_list)
        else:
            # if the status isnt listed as sold just skip this property
            # we dont want to wind up with a bunch of rentals that we cant scrape
            # or duplicates of properties we already scraped when filtering by
            # homes for sale
            pass
    except:
        return None

In [13]:
def generate_page_urls(location, num_pages, sold=False):
    """
    Description: get urls for first 20 pages for a location
    Parameter 1: location (str): e.g. 'chicago-il'
    Parameter 2: status (str): 'sold' or 'for_sale'
    
    Return: first 20 page urls (list)
    """
    
    if sold:
        base_url = 'https://www.zillow.com/{}/sold/'.format(location)
    else:
        base_url = 'https://www.zillow.com/homes/{}/'.format(location)
    
    first_20_pages_ext = [base_url + str(i) + '_p' for i in range(2,num_pages+1)]
    # as ganesh pointed out im not adding url for first page so im appending it here...
    first_20_pages_ext.append(base_url)
    return first_20_pages_ext
    

In [48]:
def get_all_property_urls(first_20_pages_ext):
    
    """
    Description: Get all the property urls for each page on Zillow
    Parameters: first 20 pages of urls (list)
    Return: urls for every property on each of the 20 pages
    """
    
    all_pages_property_urls = []
    for link in first_20_pages_ext:
        all_pages_property_urls.append(get_property_urls(link))

    # so right now i have a list of list and i want to flatten it for convenience
#     all_urls = all_pages_property_urls
    all_urls = [item for sublist in all_pages_property_urls for item in sublist]
    return all_urls

In [103]:
# cicero-il, oak-lawn-il
first_20_pages_ext = generate_page_urls('oak-lawn-il', sold=False, num_pages=7)
all_urls = get_all_property_urls(first_20_pages_ext)

In [104]:
all_urls[:10]

['https://www.zillow.com/homedetails/4001-W-106th-Pl-Oak-Lawn-IL-60453/4085154_zpid/',
 'https://www.zillow.com/homedetails/4016-W-91st-Pl-Oak-Lawn-IL-60453/4056563_zpid/',
 'https://www.zillow.com/homedetails/9528-S-Kostner-Ave-Oak-Lawn-IL-60453/4069901_zpid/',
 'https://www.zillow.com/homedetails/5100-W-96th-St-APT-220-Oak-Lawn-IL-60453/85807278_zpid/',
 'https://www.zillow.com/homedetails/6701-W-87th-Pl-Oak-Lawn-IL-60453/4062772_zpid/',
 'https://www.zillow.com/homedetails/9201-Melvina-Ave-Oak-Lawn-IL-60453/2077847003_zpid/',
 'https://www.zillow.com/homedetails/10619-Lockwood-Ct-Oak-Lawn-IL-60453/4087379_zpid/',
 'https://www.zillow.com/homedetails/5704-W-90th-St-Oak-Lawn-IL-60453/60204960_zpid/',
 'https://www.zillow.com/homedetails/10308-S-Pulaski-Rd-APT-106A-Oak-Lawn-IL-60453/4085403_zpid/',
 'https://www.zillow.com/homedetails/7100-W-95th-St-APT-303-Oak-Lawn-IL-60453/4062929_zpid/']

In [ ]:
########################## testing area ##############################

In [104]:
########################## testing end ##############################

In [105]:
# so this is where we are iterating over all the property links and calling our
# functions to scrape details from each url and return a dictionary of values
# also tracking when and on which links our functions failed to sucessfully scrape data
# so far bum links is usually only about 20 out of 800 so I'll live

data_list = []
bum_links = []
for link in all_urls:
    print(link)
    
    # change this to get property_features_sold if you want home info on properties
    # with the sold status... 
    data_raw = get_property_features(link)
    if data_raw != None:
        data_list.append(data_raw)
    else:
        # creating this so we can see what links didnt work and look into why later
        bum_links.append(link)
        

https://www.zillow.com/homedetails/4001-W-106th-Pl-Oak-Lawn-IL-60453/4085154_zpid/
[['Type', 'Single Family'], ['Year built', '1957'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '5 spaces'], ['HOA', 'None'], ['Lot', '7,993 sqft'], ['Price/sqft', '$191'], ['Total Price', '$215,000'], ['Address', '4001 W 106th Pl, Oak Lawn, IL 60453'], ['Square Feet', '1,126'], ['New construction', ' No'], ['Bedrooms', ' 3'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 2.1% higher (↑) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/4001-W-106th-Pl-Oak-Lawn-IL-60453/4085154_zpid/']]
https://www.zillow.com/homedetails/4016-W-91st-Pl-Oak-Lawn-IL-60453/4056563_zpid/
[['Type', 'Single Family'], ['Year built', 

[['Type', 'Condo'], ['Year built', '1972'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$221/mo'], ['Price/sqft', '$116'], ['Total Price', '$93,900'], ['Address', '11000 Kilpatrick Ave APT 4SE, Oak Lawn, IL 60453'], ['Square Feet', '812'], ['New construction', ' No'], ['Bedrooms', ' 2'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 55.4% lower (↓) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/11000-Kilpatrick-Ave-APT-4SE-Oak-Lawn-IL-60453/4086187_zpid/']]
https://www.zillow.com/homedetails/5718-W-88th-Pl-Oak-Lawn-IL-60453/4060432_zpid/
[['Type', 'Single Family'], ['Year built', '1957'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'No

[['Type', 'Single Family'], ['Year built', '1966'], ['Heating', 'Baseboard'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '3,659 sqft'], ['Price/sqft', '$124'], ['Total Price', '$329,900'], ['Address', '5644 W 87th Pl, Oak Lawn, IL 60453'], ['Square Feet', '2,650'], ['New construction', ' No'], ['Bedrooms', ' 5'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 56.7% higher (↑) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/5644-W-87th-Pl-Oak-Lawn-IL-60453/4060217_zpid/']]
https://www.zillow.com/homedetails/6623-W-91st-St-Oak-Lawn-IL-60453/4063343_zpid/
[['Type', 'Single Family'], ['Year built', '1987'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], 

[['Type', 'Condo'], ['Year built', '1978'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$200/mo'], ['Price/sqft', '$157'], ['Total Price', '$179,000'], ['Address', '5800 W 105th St APT 2C, Oak Lawn, IL 60453'], ['Square Feet', '1,140'], ['New construction', ' No'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 1'], ['Half bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will rise 1.3% (↑) in the next year.This home is valued 15% lower (↓) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/5800-W-105th-St-APT-2C-Oak-Lawn-IL-60453/4090385_zpid/']]
https://www.zillow.com/homedetails/8820-Mobile-Ave-APT-2A-Oak-Lawn-IL-60453/4059703_zpid/
[['Type', 'Condo'], ['Year built', '1976'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '1 spac

[['Type', 'Single Family'], ['Year built', '1999'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '6,873 sqft'], ['Price/sqft', '$183'], ['Total Price', '$374,999'], ['Address', '5241 W 108th Pl, Oak Lawn, IL 60453'], ['Square Feet', '2,048'], ['New construction', ' No'], ['Bedrooms', ' 4'], ['Bathrooms', ' 4'], ['Full bathrooms', ' 3'], ['Half bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will rise 1.3% (↑) in the next year.This home is valued 78.1% higher (↑) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/5241-W-108th-Pl-Oak-Lawn-IL-60453/4088199_zpid/']]
https://www.zillow.com/homedetails/9613-Oak-Park-Ave-Oak-Lawn-IL-60453/99358941_zpid/
[['Type', 'Single Family'], ['Year built', '2008'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['P

...

[['Type', 'Condo'], ['Year built', '1967'], ['Heating', 'No Data'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$206/month'], ['Total Price', '$69,034'], ['Address', '9820 S Pulaski Rd APT 302-2, Oak Lawn, IL 60453'], ['sqft', '--'], ['New construction', ' No'], ['Bedrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 67.2% lower (↓) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/9820-S-Pulaski-Rd-APT-302-2-Oak-Lawn-IL-60453/4071600_zpid/']]
https://www.zillow.com/homedetails/10413-Georgia-Ln-Oak-Lawn-IL-60453/4087430_zpid/
https://www.zillow.com/homedetails/9614-S-50th-Ct-Oak-Lawn-IL-60453/4068070_zpid/
https://www.zillow.com/homedetails/10210-Washington-Ave-APT-200-Oak-Lawn-IL-60453/4068721_zpid/
https://www.zillow.com/homedetail

https://www.zillow.com/homedetails/10917-S-Komensky-Ave-Oak-Lawn-IL-60453/4086851_zpid/
[['Type', 'Single Family'], ['Year built', '1954'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot', '6,873 sqft'], ['Price/sqft', '$161'], ['Total Price', '$159,000'], ['Address', '10917 S Komensky Ave, Oak Lawn, IL 60453'], ['Square Feet', '988'], ['New construction', ' No'], ['Bedrooms', ' 2'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will rise 1.3% (↑) in the next year.This home is valued 24.5% lower (↓) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/10917-S-Komensky-Ave-Oak-Lawn-IL-60453/4086851_zpid/']]
https://www.zillow.com/homedetails/9721-S-Kolmar-Ave-Oak-Lawn-IL-60453/4070031_zpid/
[['Type', 'Single Family'], ['Year built', '1967

[['Type', 'Townhouse'], ['Year built', '1968'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot', '1,886 sqft'], ['Price/sqft', '$112'], ['Total Price', '$199,900'], ['Address', '9831 S Karlov Ave, Oak Lawn, IL 60453'], ['Square Feet', '1,780'], ['New construction', ' No'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will rise 1.3% (↑) in the next year.This home is valued 5.1% lower (↓) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/9831-S-Karlov-Ave-Oak-Lawn-IL-60453/4071356_zpid/']]
https://www.zillow.com/homedetails/10225-S-Kilbourn-Ave-Oak-Lawn-IL-60453/4072193_zpid/
[['Type', 'Single Family'], ['Year built', '1962'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['

[['Type', 'Single Family'], ['Year built', '1969'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot', '0.28 Acres'], ['Price/sqft', '$108'], ['Total Price', '$419,900'], ['Address', '9736 S Kilbourn Ave, Oak Lawn, IL 60453'], ['Square Feet', '3,884'], ['New construction', ' No'], ['Bedrooms', ' 4'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 99.4% higher (↑) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/9736-S-Kilbourn-Ave-Oak-Lawn-IL-60453/4070036_zpid/']]
https://www.zillow.com/homedetails/10207-S-Tripp-Ave-Oak-Lawn-IL-60453/4072740_zpid/
[['Type', 'Single Family'], ['Year built', '1951'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot

[['Type', 'Townhouse'], ['Year built', '1995'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['Lot', '1,958 sqft'], ['Price/sqft', '$132'], ['Total Price', '$211,999'], ['Address', '10948 Jodan Dr, Oak Lawn, IL 60453'], ['Square Feet', '1,610'], ['New construction', ' No'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1'], ['neighborhood_stats', 'Home values in 60453 have risen 1.3\u200a% (↑) over the past 12 months.Zillow predicts the home values in 60453 will increase 1.3% (↑) in the next year.This home is valued 0.7% higher (↑) than the median home in 60453.The median Zestimate® for this neighborhood is $210,544.'], ['URL', 'https://www.zillow.com/homedetails/10948-Jodan-Dr-Oak-Lawn-IL-60453/4088357_zpid/']]
https://www.zillow.com/homedetails/9001-S-49th-Ct-Oak-Lawn-IL-60453/4057787_zpid/
[['Type', 'Single Family'], ['Year built', '1955'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['L

In [17]:
# pd.DataFrame(data_list)

,Type,Year built,Heating,Cooling,Parking,Lot,Price/sqft,Total Price,Address,Square Feet,New construction,Bedrooms,Bathrooms,Full bathrooms,Half bathrooms,neighborhood_stats,URL,HOA,sqft
0,Single Family,1968,"Forced air, Gas",Central,2 spaces,"10,018 sqft",$191,"$470,000","928 Royal Blackheath Ct, Naperville, IL 60563","2,464",No,4,3,2,1,Home values in 60563 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/928-Royal-B...,NaN,NaN
1,Single Family,1999,"Forced air, Gas",Central,3 spaces,0.29 Acres,$160,"$529,900","1452 Chelsea Ln, Naperville, IL 60565","3,305",No,4,4,3,1,Home values in 60565 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/1452-Chelse...,NaN,NaN
2,Condo,1989,Forced air,Central,1 space,NaN,$187,"$214,500","1292 Rhodes Ln #1292, Naperville, IL 60540","1,149",No,2,2,2,NaN,Home values in 60540 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/1292-Rhodes...,$238/month,NaN
3,Single Family,1915,Gas,No Data,1 space,"10,018 sqft",$626,"$349,900","1131 N Washington St, Naperville, IL 60563",559,No,1,1,1,NaN,Home values in 60563 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/1131-N-Wash...,NaN,NaN
4,Single Family,1986,No Data,No Data,2 spaces,0.35 Acres,$217,"$739,900","512 Braemar Ave, Naperville, IL 60563","3,406",No,5,5,4,1,Home values in 60563 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/512-Braemar...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,Single Family,1921,Baseboard,Central,1 space,"8,372 sqft",$222,"$599,900","215 S Columbia St, Naperville, IL 60540","2,702",No,4,4,4,NaN,Home values in 60540 have risen 1.0 % (↑) over...,https://www.zillow.com/homedetails/215-S-Colum...,NaN,NaN
373,Townhouse,2020,No Data,No Data,2 spaces,No Data,$226,"$494,588","Buildable plan: Metro, Columbia Park Townes, N...",NaN,Yes,3,3,2,1,Home values in 60563 have risen 1.0 % (↑) over...,https://www.zillow.com/community/columbia-park...,NaN,"2,185"
374,Single Family,2020,No Data,No Data,2 spaces,No Data,$160,"$540,990","Buildable plan: Castleton, Wagner Farms, Naper...",NaN,Yes,4,3,2,1,Home values in 60564 have risen 1.0 % (↑) over...,https://www.zillow.com/community/wagner-farms/...,NaN,"3,391"
375,Single Family,2020,Forced air,Central,3 spaces,No Data,$305,"$901,168","Buildable plan: The Abbey, DJK Custom Homes of...",NaN,Yes,4,3,2,1,Home values in 60540 have risen 1.0 % (↑) over...,https://www.zillow.com/community/djk-custom-ho...,NaN,"2,959"


In [106]:
# data_list
df = pd.DataFrame(data_list)
# df.to_csv('sold_raw.csv', header=True)

In [107]:
df.drop(columns=['sqft'], inplace=True)

In [108]:
df.columns.to_list()

['Type',
 'Year built',
 'Heating',
 'Cooling',
 'Parking',
 'HOA',
 'Lot',
 'Price/sqft',
 'Total Price',
 'Address',
 'Square Feet',
 'New construction',
 'Bedrooms',
 'Bathrooms',
 'Full bathrooms',
 'neighborhood_stats',
 'URL',
 'Half bathrooms']

In [109]:
df.Address

0              4001 W 106th Pl, Oak Lawn, IL 60453
1               4016 W 91st Pl, Oak Lawn, IL 60453
2           9528 S Kostner Ave, Oak Lawn, IL 60453
3       5100 W 96th St APT 220, Oak Lawn, IL 60453
4               6701 W 87th Pl, Oak Lawn, IL 60453
                          ...                     
180    9440 S 51st Ave APT 502, Oak Lawn, IL 60453
181          9246 S Keeler Ave, Oak Lawn, IL 60453
182     4929 W 87th St APT 3NW, Oak Lawn, IL 60453
183             9117 S 53rd Ct, Oak Lawn, IL 60453
184     4003 W 105th St APT 3W, Oak Lawn, IL 60453
Name: Address, Length: 185, dtype: object

In [110]:
df.to_csv('oak_lawn_il.csv')